<a href="https://colab.research.google.com/github/guidobarra/pyhton-GPU/blob/main/BarraQuelca_GuidoAlberto_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1 Introducción

El siguiente ejemplo se realizara la busqueda de un elemento dentro de un array.
En el mundo de la informatica hay varios algoritmos de busqueda como por ejemplo la busqueda binaria en el cual los elementos del array deben estar ordenados para realizar el algoritmo, otro algoritmo es la busqueda secuencial el mas simple de todos. La busqueda de un objeto o palabra es algo muy utlizado, un ejemplo es el algoritmo de busqueda de google, buscar una palabra o secuencia de caracteres dentro de un archivo, buscar claves dentro de una tabla de base de datos.

EL objetivo es enseñar el funcionamiento del Lenguaje Python, CUDA y buscar un numero dentro de un array. El ejemplo es ilustrativo para entender los multi hilos de la GPU de una dimencion y ver la performance utilizando la GPU y CPU.

---
# 2 Armado del ambiente
Toma los parametros de elementos del array y el numero que se debe buscar, lo deja disponible al contexto de ejecuciòn del cuaderno colab.

In [15]:
#@title # 2.1 Parámetros de ejecución
#@markdown ### Especifique las dimeciones de la matrices:

try: 
  cant_element =  100000000#@param {type:"integer"}
  encontrar_numero =  1000000000#@param {type:"integer"}
except Exception as e:
  print("Error de ingresar los parametros")
  print("Error debido a: ", e.__class__)
  print(e)






---
## 2.2 Validar las parametros

In [5]:

if cant_element < 0 :
  print("ERROR LA CANTIDAD DE ELEMENTOS NO PUEDE SER MENOR QUE CERO")

---
## 2.3 Instala en el cuaderno el módulo CUDA de Python.

In [17]:
!pip install pycuda

---
# 3 Desarrollo
Ejecución del algoritmo de busqueda utilizando la CPU para el desarrollo en su forma secuencual y la GPU para el desarrollo en su forma paralela.

## 3.1 Desarrollo Secuencial
Ejecución del algoritmo de busqueda de forma secuencial utilizando la CPU.

In [10]:
try:
  %matplotlib inline
  from datetime import datetime
  tiempo_total = datetime.now()
  import numpy

  # --------------------------------------------
  # Definición de función que transforma el tiempo en  milisegundos
  tiempo_en_ms = lambda dt:(dt.days * 24 * 60 * 60 + dt.seconds) * 1000 + dt.microseconds / 1000.0
  # --------------------------------------------

  tiempo_definicion_vector = datetime.now()

  # CPU - Defino la memoria del vector de n elementos en cpu.
  vector = numpy.array(range(-cant_element, cant_element, 2))
  vector = vector.astype(numpy.int32())

  tiempo_definicion_vector = datetime.now() - tiempo_definicion_vector

  tiempo_find_secuencial = datetime.now()

  position = -1
  for i in range(cant_element):
    if vector[i] == encontrar_numero:
      position = i
      break

  tiempo_find_secuencial = datetime.now() - tiempo_find_secuencial

  tiempo_total = datetime.now() - tiempo_total

  print("\n**ENCONTRAR UN ELEMENTO DENTRO DE UN VECTOR UTILIZANDO LA CPU**\n")
  print("Cantidad de elementos del vector: ", cant_element)
  print("Numero a encontrar: ", encontrar_numero)
  print("Vector: \n", vector)
  if position == -1:
    print("No se encontro el numero: ", encontrar_numero)
  else:
    print("Se encontro el numero: ", encontrar_numero)
    print("Posicion dentro del vector es: ", position)

  print("Tiempo de definicion del vector: ", tiempo_en_ms( tiempo_definicion_vector ), "[ms]" )
  print("Tiempo CPU  : ", tiempo_en_ms( tiempo_find_secuencial ), "[ms]" )
  print("Tiempo TOTAL: ", tiempo_en_ms( tiempo_total ), "[ms]" )
except Exception as e:
  print("Oops Ocurrio una error!")
  print("Error debido a: ", e.__class__)
  print(e)


**ENCONTRAR UN ELEMENTO DENTRO DE UN VECTOR UTILIZANDO LA CPU**

Cantidad de elementos del vector:  1000000
Numero a encontrar:  10000000
Vector: 
 [-1000000  -999998  -999996 ...   999994   999996   999998]
No se encontro el numero:  10000000
Tiempo de definicion del vector:  201.177 [ms]
Tiempo CPU  :  1624.161 [ms]
Tiempo TOTAL:  1825.374 [ms]


## 3.2 Desarrollo Paralelo
Ejecución del algoritmo de busqueda de forma paralelo utilizando la GPU.

In [16]:
try:
  %matplotlib inline
  from datetime import datetime
  tiempo_total = datetime.now()
  
  import numpy
  import pycuda.driver as cuda
  import pycuda.autoinit
  from   pycuda.compiler import SourceModule
  
  # --------------------------------------------
  # Definición de función que transforma el tiempo en  milisegundos
  tiempo_en_ms = lambda dt:(dt.days * 24 * 60 * 60 + dt.seconds) * 1000 + dt.microseconds / 1000.0
  # --------------------------------------------
  
  tiempo_definicion_vector = datetime.now()
  
  # CPU - Defino la memoria del vector de n elementos en cpu.
  vector = numpy.array(range(-cant_element, cant_element, 2))
  vector = vector.astype(numpy.int32())
  
  # CPU - Defino la memoria del vector position_v de 1 elemento en cpu.
  position_v    = numpy.random.randn(1)*0
  position_v[0] = -1
  position_v    = position_v.astype(numpy.int32())
  
  tiempo_definicion_vector = datetime.now() - tiempo_definicion_vector
  
  tiempo_reserva_memoria_GPU = datetime.now()
  # CPU - reservo la memoria GPU.
  vector_Gpu = cuda.mem_alloc(vector.nbytes)
  position_v_Gpu = cuda.mem_alloc(position_v.nbytes)
  tiempo_reserva_memoria_GPU = datetime.now() - tiempo_reserva_memoria_GPU
  
  tiempo_copia_memoria_GPU = datetime.now()
  # GPU - Copio la memoria al GPU.
  cuda.memcpy_htod(vector_Gpu, vector)
  cuda.memcpy_htod(position_v_Gpu, position_v)
  tiempo_copia_memoria_GPU = datetime.now() - tiempo_copia_memoria_GPU
  
  #CPU - Defino la funcion kernel que ejecutará en GPU
  module = SourceModule("""
  __global__ void find(int tam, int number_find, int *vector, int *position)
  {
      // Calculo las coordenadas del Thread en dos dimensiones.
      int idx = threadIdx.x + blockIdx.x*blockDim.x;
  
      // Verifico que los Thread, esten dentro de las dimensiones del vector
      if( idx < tam ) {
        if (vector[idx] == number_find && position[0] == -1) {
          position[0] = idx;
        }
      }
  }
  
  """)
  
  # CPU - Genero la función kernel.
  kernel = module.get_function("find")
  
  # GPU - Ejecuta el kernel.
  # TODO: Falta consultar limites del GPU, para armar las dimensiones correctamente.
  dim_hilo = 256
  dim_bloque = numpy.int( (cant_element+dim_hilo-1) / dim_hilo )
  print( "Thread x: ", dim_hilo, ", Bloque x:", dim_bloque )
  
  tiempo_find_paralelo = datetime.now()
  
  kernel( numpy.int32(cant_element),
          numpy.int32(encontrar_numero),
          vector_Gpu,
          position_v_Gpu,
          block=( dim_hilo, 1, 1 ),
          grid=(dim_bloque, 1,1) )
  
  tiempo_find_paralelo = datetime.now() - tiempo_find_paralelo
  
  # GPU - Copio el resultado desde la memoria GPU.
  cuda.memcpy_dtoh( position_v, position_v_Gpu )
  tiempo_total = datetime.now() - tiempo_total
  
  print("\n**ENCONTRAR UN ELEMENTO DENTRO DE UN VECTOR UTILIZANDO LA GPU**\n")
  print("Cantidad de elementos del vector: ", cant_element)
  print("Numero a encontrar: ", encontrar_numero)
  print("Vector: \n", vector)
  if position == -1 or position_v[0] == -1:
    print("No se encontro el numero: ", encontrar_numero)
  else:
    print("Se encontro el numero: ", encontrar_numero)
    print("Posicion dentro del vector es: ", position_v[0])
    print("Posicion dentro del vector es: ", position)

  print("Tiempo de definicion del vector: ", tiempo_en_ms( tiempo_definicion_vector ), "[ms]" )
  print("Tiempo de reserva del vector: ", tiempo_en_ms( tiempo_reserva_memoria_GPU ), "[ms]" )
  print("Tiempo de copia del vector: ", tiempo_en_ms( tiempo_copia_memoria_GPU ), "[ms]" )
  print("Tiempo GPU  : ", tiempo_en_ms( tiempo_find_paralelo ), "[ms]" )
  print("Tiempo TOTAL: ", tiempo_en_ms( tiempo_total ), "[ms]" )
except Exception as e:
  print("Oops Ocurrio una error!")
  print("Error debido a: ", e.__class__)
  print(e)

Thread x:  256 , Bloque x: 390625

**ENCONTRAR UN ELEMENTO DENTRO DE UN VECTOR UTILIZANDO LA GPU**

Cantidad de elementos del vector:  100000000
Numero a encontrar:  1000000000
Vector: 
 [-100000000  -99999998  -99999996 ...   99999994   99999996   99999998]
No se encontro el numero:  1000000000
Tiempo de definicion del vector:  19262.561 [ms]
Tiempo de reserva del vector:  1.06 [ms]
Tiempo de copia del vector:  88.149 [ms]
Tiempo GPU  :  0.093 [ms]
Tiempo TOTAL:  19356.058 [ms]


---
# 4 Tabla de pasos


 Procesador | Funciòn | Detalle
------------|---------|----------
CPU      | pip install pycuda    | Instala en el cuaderno los driver de CUDA para Python.
CPU      |  import                | Importa los módulos para funcionar.
CPU      |  datetime.now()        | Toma el tiempo actual.
CPU      |  numpy.random(n)       | Crear una vector con numeros ramdoms de n elementos.
CPU      |  ar.astype(np.int32()) | Convierte los elemementos del array ar en elementos de tipo int 32 bit.
**GPU**  |  cuda.mem_alloc()      | Reserva la memoria para el vector en la GPU.
**GPU**  |  cuda.memcpy_htod()    | Copia los valores en crudo del vector al GPU.
CPU      |  SourceModule()        | Posee el còdigo del kernel.
CPU      |  module.get_function() | convierte el texto del kernel en funcion de Python.
CPU      |  dim_hilo_x            | Calcula la dimension para la ejecuciòn de 1D.
**GPU**  |  kernel()              | Ejecuta el kernel en GPU, enviando los parametros.
CPU      |  print()               | Informa medieante un mensaje.
CPU      | cuda.memcpy_dtoh()     | Copia desde la memoria GPU al CPU.



---
# 5 Conclusiones

Se hicieron varias pruebas con vectores de un tamaño pequeño (50 elementos), y con con un tamaño grande (100000000 elementos). Se observo que para vectores pequeños el tiempo de ejecucion de la forma secuencial utilizando la CPU y el tiempo de ejecucion de la forma paralela utilizando la GPU, son aproximadamente iguales y en ocaciones el tiempo de ejecucion total de la GPU es mayor que el tiempo de ejecucion total de la CPU, pareceria que no estaria mejorando el tiempo de ejecucion utilizando la GPU. Sin embargo al momento de tener vectores con muchos elementos se observo que el tiempo de ejecucion de la busqueda de un elemento utilizando la GPU se mantiene constante, aproximadamente, mientras que el tiempo de ejecucion de la busqueda de un elemento utilizando la CPU aumenta significativamente cada vez que aumenta la cantidad de elementos del vector.
Se concluye que la utilizacion de la GPU baja el tiempo de ejecucion de la busqueda de un elemento en comparacion con la CPU siempre que se tiene un vector de tamaño considerable por ejemplo 100.

---
# 6 Bibliografía

[1] Desarrollo Teorico algoritmo de busqueda: [Pagina Wiki](https://es.wikipedia.org/wiki/Algoritmo_de_b%C3%BAsqueda#:~:text=Un%20algoritmo%20de%20b%C3%BAsqueda%20es,en%20una%20partida%20de%20ajedrez.)

[2] Introducción a Python: [Página Colab](https://github.com/wvaliente/SOA_HPC/blob/main/Documentos/Python_Basico.ipynb) 

[3] Algoritmo de Busqueda y Ordenamiento: [PDF](https://www.inf.utfsm.cl/~noell/IWI-131-p1/Tema8b.pdf)
